In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,auc,average_precision_score,balanced_accuracy_score,brier_score_loss,classification_report,cohen_kappa_score,confusion_matrix,dcg_score,det_curve,f1_score,fbeta_score,hamming_loss,hinge_loss,jaccard_score,log_loss,matthews_corrcoef,multilabel_confusion_matrix,ndcg_score,precision_recall_curve,precision_recall_fscore_support,precision_score,recall_score,roc_auc_score,roc_curve,top_k_accuracy_score,zero_one_loss
from sklearn.model_selection import train_test_split
from scipy.misc import electrocardiogram
import pywt
import scipy.signal as signal
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp
from torchmetrics.audio import SignalDistortionRatio

In [2]:
hemiplegiaFiles = os.listdir('../Data/Hemiplegia')
normalFiles = os.listdir('../Data/Normal')
hemiplegia = pd.DataFrame()
normal = pd.DataFrame()
for i in hemiplegiaFiles:
    df = pd.read_csv('../Data/Hemiplegia/' + i)
    df.columns = ['index','Ax','Ay','Az','Gx','Gy','Gz']
    hemiplegia = pd.concat([hemiplegia,df],axis=0)
for i in normalFiles:
    df = pd.read_csv('../Data/Normal/' + i)
    df.columns = ['index','Ax','Ay','Az','Gx','Gy','Gz']
    normal = pd.concat([normal,df],axis=0)
hemiplegia = hemiplegia.drop(columns='index')
hemiplegia['Diagnosis'] = 'Hemiplegia'
normal = normal.drop(columns='index')
normal['Diagnosis'] = 'Normal'
combined = pd.concat([normal,hemiplegia],axis=0)

In [3]:
cols = ['Ax','Ay','Az','Gx','Gy','Gz']
dfDWT = pd.DataFrame()
dfButterworth = pd.DataFrame()
dfGeneralDWT = pd.DataFrame()
data = normal
for col in cols:
    wavelet_type='db6'
    DWTcoeffs = pywt.wavedec(data[col],wavelet_type,mode='symmetric', level=9, axis=-1)
    DWTcoeffs[-1] = np.zeros_like(DWTcoeffs[-1])
    DWTcoeffs[-2] = np.zeros_like(DWTcoeffs[-2])
    DWTcoeffs[-3] = np.zeros_like(DWTcoeffs[-3])
    DWTcoeffs[-4] = np.zeros_like(DWTcoeffs[-4])
    DWTcoeffs[-5] = np.zeros_like(DWTcoeffs[-5])
    DWTcoeffs[-6] = np.zeros_like(DWTcoeffs[-6])
    DWTcoeffs[-7] = np.zeros_like(DWTcoeffs[-7])
    DWTcoeffs[-8] = np.zeros_like(DWTcoeffs[-8])
    DWTcoeffs[-9] = np.zeros_like(DWTcoeffs[-9])
    filtered_data_dwt=pywt.waverec(DWTcoeffs,wavelet_type,mode='symmetric',axis=-1) 
    fc = 0.9  # Cut-off frequency of the butterworth filter
    w = fc / (100 / 2) # Normalize the frequency
    b, a = signal.butter(5, w, 'low')
    filtered_data_butterworth = signal.filtfilt(b, a, data[col])
    tempDWT = pd.DataFrame(filtered_data_dwt)
    tempDWT.columns = [col]
    dfDWT = pd.concat([dfDWT,tempDWT],axis=1)
    tempButterworth = pd.DataFrame(filtered_data_butterworth)
    tempButterworth.columns = [col]
    dfButterworth = pd.concat([dfButterworth,tempButterworth],axis=1)
    genDWTcoeffs = pywt.wavedec(data[col],wavelet_type,mode='symmetric', level=9, axis=-1)
    filtered_data_gen_dwt=pywt.waverec(genDWTcoeffs,wavelet_type,mode='symmetric',axis=-1)
    tempgenDWT = pd.DataFrame(filtered_data_gen_dwt)
    tempgenDWT.columns = [col]
    dfGeneralDWT = pd.concat([dfGeneralDWT,tempgenDWT],axis=1)
    
dfDWT[['Diagnosis']] = 'Normal'
dfButterworth[['Diagnosis']] = 'Normal'
dfDWTNormal = dfDWT
dfButterworthNormal = dfButterworth
dfGeneralDWT[['Diagnosis']] = 'Normal'
generalDWT = dfGeneralDWT



cols = ['Ax','Ay','Az','Gx','Gy','Gz']
dfDWT = pd.DataFrame()
dfButterworth = pd.DataFrame()
dfGeneralDWT = pd.DataFrame()
data = hemiplegia
for col in cols:
    wavelet_type='db6'
    DWTcoeffs = pywt.wavedec(data[col],wavelet_type,mode='symmetric', level=9, axis=-1)
    DWTcoeffs[-1] = np.zeros_like(DWTcoeffs[-1])
    DWTcoeffs[-2] = np.zeros_like(DWTcoeffs[-2])
    DWTcoeffs[-3] = np.zeros_like(DWTcoeffs[-3])
    DWTcoeffs[-4] = np.zeros_like(DWTcoeffs[-4])
    DWTcoeffs[-5] = np.zeros_like(DWTcoeffs[-5])
    DWTcoeffs[-6] = np.zeros_like(DWTcoeffs[-6])
    DWTcoeffs[-7] = np.zeros_like(DWTcoeffs[-7])
    DWTcoeffs[-8] = np.zeros_like(DWTcoeffs[-8])
    DWTcoeffs[-9] = np.zeros_like(DWTcoeffs[-9])
    filtered_data_dwt=pywt.waverec(DWTcoeffs,wavelet_type,mode='symmetric',axis=-1) 
    fc = 0.9  # Cut-off frequency of the butterworth filter
    w = fc / (100 / 2) # Normalize the frequency
    b, a = signal.butter(5, w, 'low')
    filtered_data_butterworth = signal.filtfilt(b, a, data[col])
    tempDWT = pd.DataFrame(filtered_data_dwt)
    tempDWT.columns = [col]
    dfDWT = pd.concat([dfDWT,tempDWT],axis=1)
    tempButterworth = pd.DataFrame(filtered_data_butterworth)
    tempButterworth.columns = [col]
    dfButterworth = pd.concat([dfButterworth,tempButterworth],axis=1)
    genDWTcoeffs = pywt.wavedec(data[col],wavelet_type,mode='symmetric', level=9, axis=-1)
    filtered_data_gen_dwt=pywt.waverec(genDWTcoeffs,wavelet_type,mode='symmetric',axis=-1)
    tempgenDWT = pd.DataFrame(filtered_data_gen_dwt)
    tempgenDWT.columns = [col]
    dfGeneralDWT = pd.concat([dfGeneralDWT,tempgenDWT],axis=1)
dfDWT[['Diagnosis']] = 'Hemiplegia'
dfButterworth[['Diagnosis']] = 'Hemiplegia'
dfDWThemiplegia = dfDWT
dfButterworthHemiplegia = dfButterworth
dfGeneralDWT[['Diagnosis']] = 'Hemiplegia'
generalDWTHemi = dfGeneralDWT

combinedDWT = pd.concat([dfDWTNormal,dfDWThemiplegia],axis=0).sample(frac=1).reset_index(drop=True)
combinedButterworth = pd.concat([dfButterworthNormal,dfButterworthHemiplegia],axis=0).sample(frac=1).reset_index(drop=True)
combinedDWTgen = pd.concat([generalDWT,generalDWTHemi],axis=0).sample(frac=1).reset_index(drop=True)

In [ ]:
plt.figure(1)
plt.psd(data['Ay'], NFFT=512, Fs=200, label='original data', color='blue')
plt.psd(filtered_data_dwt, NFFT=512, Fs=200, color='red', label='filtered data (DWT)')
plt.psd(filtered_data_butterworth, NFFT=512, Fs=200, color='black', label='filtered data (Butterworth)')
plt.legend()

plt.figure(2)
plt.subplot(311)
plt.plot(data['Ay'],label='original data', color='blue')
plt.title('original')
plt.subplot(312)
plt.plot(filtered_data_dwt, color='red', label='filtered data (DWT)')
plt.title('filtered (DWT)')
plt.subplot(313)
plt.plot(filtered_data_butterworth, color='black', label='filtered data (Butterworth)')
plt.title('filtered (Butterworth)')

In [5]:
combinedDWT = combinedDWT[1:].reset_index(drop=True)

In [6]:
combinedButterworth

,Ax,Ay,Az,Gx,Gy,Gz,Diagnosis
0,0.080345,10.913272,2.237133,-0.174711,0.274580,0.102331,Normal
1,0.089161,10.837887,2.225845,-0.166107,0.261892,0.104865,Normal
2,0.097334,10.763483,2.214429,-0.157476,0.249378,0.107063,Normal
3,0.104832,10.690148,2.202896,-0.148826,0.237057,0.108919,Normal
4,0.111625,10.617968,2.191260,-0.140170,0.224948,0.110423,Normal
...,...,...,...,...,...,...,...
28607,-0.068065,9.269927,2.013755,-0.106107,0.448504,0.367106,Hemiplegia
28608,-0.067893,9.269787,2.014056,-0.106245,0.448750,0.367171,Hemiplegia
28609,-0.067756,9.269672,2.014302,-0.106357,0.448947,0.367221,Hemiplegia
28610,-0.067649,9.269581,2.014501,-0.106448,0.449103,0.367260,Hemiplegia


In [7]:
combined

,Ax,Ay,Az,Gx,Gy,Gz,Diagnosis
0,-0.095768,10.975020,2.298433,-0.143920,0.281749,0.107303,Normal
1,-0.095768,10.975020,2.298433,-0.019740,0.199509,0.170187,Normal
2,0.545878,9.988609,3.648763,-0.019740,0.199509,0.170187,Normal
3,0.545878,9.988609,3.648763,-0.047839,0.107617,0.218969,Normal
4,1.675941,9.950302,2.700659,-0.047839,0.107617,0.218969,Normal
...,...,...,...,...,...,...,...
1596,-0.574608,8.973468,0.852336,0.208514,-1.026934,-0.768887,Hemiplegia
1597,-0.574608,8.973468,0.852336,0.272027,-0.676018,-0.545276,Hemiplegia
1598,-0.746991,8.772355,1.264139,0.272027,-0.676018,-0.545276,Hemiplegia
1599,-0.746991,8.772355,1.264139,0.322258,-0.459772,-0.385456,Hemiplegia


In [8]:
restored = combinedDWT[['Ax','Ay','Az','Gx','Gy','Gz']].to_numpy()
real = combined[['Ax','Ay','Az','Gx','Gy','Gz']].to_numpy()
metrics = pd.DataFrame()
values = [{
        "Name": 'Raw v/s DWT',
        "MSE": np.round(mse(restored,real),4),
        "RMSE": np.round(rmse(restored, real),4),
        "SCC": np.round(scc(restored, real),4),
        "SAM": np.round(sam(restored, real),4)
    }]
restored = combinedButterworth[['Ax','Ay','Az','Gx','Gy','Gz']].to_numpy()
real = combined[['Ax','Ay','Az','Gx','Gy','Gz']].to_numpy()
values.append(
    {"Name": 'Raw v/s BW',
        "MSE": np.round(mse(restored,real),4),
        "RMSE": np.round(rmse(restored, real),4),
        "SCC": np.round(scc(restored, real),4),
        "SAM": np.round(sam(restored, real),4)
    })
restored = combinedButterworth[['Ax','Ay','Az','Gx','Gy','Gz']].to_numpy()
real = combinedDWT[['Ax','Ay','Az','Gx','Gy','Gz']].to_numpy()
values.append(
    {"Name": 'DWT v/s BW',
        "MSE": np.round(mse(restored,real),4),
        "RMSE": np.round(rmse(restored, real),4),
        "SCC": np.round(scc(restored, real),4),
        "SAM": np.round(sam(restored, real),4)
    })
metrics = pd.concat([metrics,pd.DataFrame.from_dict(values)])
metrics

,Name,MSE,RMSE,SCC,SAM
0,Raw v/s DWT,1.2354,1.1115,0.9792,0.2747
1,Raw v/s BW,1.1405,1.0680,0.9804,0.2636
2,DWT v/s BW,0.0441,0.2101,0.9993,0.0532


In [75]:
metrics.to_csv('Signal transformation difference.csv',index=False)

In [77]:
combinedDWT.to_csv('Combined DWT.csv',index=False)
combinedButterworth.to_csv('Combined Butterworth.csv',index=False)
combined.to_csv('Combined.csv',index=False)

In [12]:
pywt.wavedec(data[col],wavelet_type,mode='symmetric', level=9, axis=-1)

10

In [ ]:
plt.rcParams.update({'font.size':16})
dfDWTNormal[['Gy']].iloc[100:30000].plot(ylim=(-1,1),grid=True,ylabel='Magnitude $\longrightarrow$',xlabel='Sequence $\longrightarrow$', fontsize=16, logx=True)
dfDWThemiplegia[['Gy']].iloc[100:30000].plot(ylim=(-1,1),grid=True,ylabel='Magnitude $\longrightarrow$',xlabel='Sequence $\longrightarrow$', fontsize=16, logx=True)